In [17]:
import os
import re
import glob
import os
import sys
import ast
import os.path as op
from collections import defaultdict
from copy import deepcopy
import copy

import pickle
import mne
import scipy
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import set_config
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.feature_selection import SelectKBest, SelectPercentile, f_regression, SelectFpr, SelectFdr, SequentialFeatureSelector
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn import datasets, linear_model

warnings.filterwarnings("ignore")

sns.set_theme(style="whitegrid", palette="deep")


In [18]:
#create dataframe from participants.tsv containing columns 1 and 2
df = pd.read_csv('../ds003944/participants.tsv', sep='\t', usecols=[0,1])
print(df)


   participant_id       type
0        sub-1448    Control
1        sub-1824  Psychosis
2        sub-1971    Control
3        sub-1983  Psychosis
4        sub-1989    Control
..            ...        ...
77      sub-2184A  Psychosis
78      sub-2193A    Control
79      sub-2214A    Control
80      sub-2217A  Psychosis
81      sub-2221A    Control

[82 rows x 2 columns]


In [19]:
#add mne object to dataframe from vmrk files within folder that name equals value in column 1
df['psyhosis_raw_data'] = [mne.io.read_raw_brainvision('../ds003944/'+str(i)+'/eeg/'+str(i)+'_task-Rest_eeg.vhdr', preload=True) for i in df['participant_id']]

Extracting parameters from ../ds003944/sub-1448/eeg/sub-1448_task-Rest_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 305999  =      0.000 ...   305.999 secs...
Extracting parameters from ../ds003944/sub-1824/eeg/sub-1824_task-Rest_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 300999  =      0.000 ...   300.999 secs...
Extracting parameters from ../ds003944/sub-1971/eeg/sub-1971_task-Rest_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 300999  =      0.000 ...   300.999 secs...
Extracting parameters from ../ds003944/sub-1983/eeg/sub-1983_task-Rest_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 300999  =      0.000 ...   300.999 secs...
Extracting parameters from ../ds003944/sub-1989/eeg/sub-1989_task-Rest_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 309999  =      0.000 ...   309.999 secs...
Extracting parameters from ../ds003944/sub-1990/eeg/sub-1990_task-Rest_eeg.vhdr...
Setting channel info structure...
Reading 0 ...

In [20]:
dir_with_data = "../ds003944/preprocessed-data-autoreject/"
# df.insert(3, "processed_frequencies", False)

FREQ_BANDS = {    "theta": [4.5, 8.5],
                  "alpha": [8.5, 11.5],
                  "delta": [0.5, 4.5],
                  "beta": [15.5,30]}


def eeg_power_band(epochs, FREQ_BANDS):
    """EEG relative power band feature extraction.

    This function takes an ``mne.Epochs`` object and creates EEG features based
    on relative power in specific frequency bands that are compatible with
    scikit-learn.

    Parameters
    ----------
    epochs : Epochs
        The data.

    Returns
    -------
    X : numpy array of shape [n_samples, 5]
        Transformed data.
    """

    spectrum = epochs.compute_psd(picks='eeg', fmin=0.5, fmax=30.)
    psds, freqs = spectrum.get_data(return_freqs=True)
    # Normalize the PSDs
    psds /= np.sum(psds, axis=-1, keepdims=True)

    X = []
    for fmin, fmax in FREQ_BANDS.values():
        psds_band = psds[:, (freqs >= fmin) & (freqs < fmax)].mean(axis=-1)
        X.append(psds_band.reshape(len(psds), -1))

    return np.concatenate(X, axis=1)

    
''' uwaga! poniżej jest try i except, żeby nie zatrzymywało się na błędzie, że nie ma pliku dla jakiegoś uczestnika (części plików nie ma, dla tych uczestników dla których był za bardzo zaszumiony sygnał'''
''' żeby przetestować czy wszystko działa i nie wywala błędów już na etapie pierwszegu uczestnika trzeba zakomentować to try i except'''

for sub_id in df['participant_id'].values:
    try:
        mat_epochs_readed = mne.read_epochs(f"{dir_with_data}{sub_id}-epo.fif")
        evoked = mat_epochs_readed.average()
        evoked_channels = evoked.pick_channels(['F7', 'F5', 'F3', 'F1', 'F2', 'F4', 'F6', 'F8', 'P7', 'P5', 'P3', 'P1', 'P2', 'P4', 'P6', 'P8', 'O1', 'O2', 'C5', 'C3', 'C1', 'C2', 'C4', 'C6', 'T9', 'T7', 'T8', 'T10'])
        print(eeg_power_band(evoked_channels, FREQ_BANDS))
        ''' tak jak widać liczy się to co powinno i printuje na ekran, ale nie umiem tego zapisać do data frame'a - powinno iść do nowej kolumny, w tym wierszu dla odpowiedniego uczestnika (aktualne sub_id)'''
        '''poniżej moje nieudolne próby zrobienia tego, które wywalały błąd'''
        # df.loc[df['participant_id'] == sub_id, 'processed_frequencies'] = eeg_power_band(evoked_channels, FREQ_BANDS)
        #df['processed_frequencies'] = df['participant_id'].where(df['participant_id']==sub_id, eeg_power_band(evoked_channels, FREQ_BANDS))
    except:
        print("file for "+sub_id+" doesn't exist")
        
    

Reading c:\dev\python\eggML-project\..\ds003944\preprocessed-data-autoreject\sub-1448-epo.fif ...
    Found the data of interest:
        t =       0.00 ...   29999.00 ms
        0 CTF compensation matrices available
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
[[1.45529922e-03 1.15025528e-03 4.94842247e-03 1.73288509e-04]
 [1.54448470e-03 1.04238453e-03 4.85690224e-03 1.90116130e-04]
 [1.71266784e-03 1.08297761e-03 4.75599524e-03 1.41082397e-04]
 [1.72194567e-03 9.55070605e-04 5.01851337e-03 1.05771997e-04]
 [1.64576234e-03 8.96185620e-04 5.19765472e-03 1.03815002e-04]
 [1.69206569e-03 9.36585914e-04 4.99292615e-03 1.36912477e-04]
 [1.63539589e-03 9.85312388e-04 4.75431336e-03 2.09885077e-04]
 [1.49326204e-03 8.16464734e-04 5.13885475e-03 2.01848853e-04]
 [9.11454231e-04 8.95119678e-04 5.94297856e-03 1.14617695e-04]
 [1.13827235e-03 1.02333310e-03 5.51769476e-03 1.

In [21]:
print(eeg_power_band(evoked_channels, FREQ_BANDS))

    Using multitaper spectrum estimation with 7 DPSS windows
[[0.00090103 0.00071816 0.00586072 0.00018665]
 [0.00106514 0.00085522 0.00564411 0.00016152]
 [0.00117644 0.00092095 0.00561592 0.000122  ]
 [0.00122681 0.00089146 0.00564094 0.00010653]
 [0.0010839  0.00085451 0.00579295 0.00011019]
 [0.00102446 0.00091362 0.0056625  0.00014455]
 [0.00090879 0.00078691 0.00589867 0.00015985]
 [0.00076206 0.00067794 0.00619288 0.0001591 ]
 [0.00144584 0.00105411 0.00436216 0.0002918 ]
 [0.0013937  0.00108778 0.00463057 0.00025062]
 [0.00154406 0.00141901 0.00435304 0.00022311]
 [0.00142934 0.00121464 0.0048872  0.00017393]
 [0.00146359 0.00100174 0.00502039 0.00015304]
 [0.00123126 0.00099458 0.00516991 0.00017742]
 [0.00131287 0.00126205 0.00482354 0.00020521]
 [0.00127453 0.00135522 0.00468094 0.00022335]
 [0.00113584 0.00109647 0.00508921 0.00022476]
 [0.00114533 0.00096488 0.0051361  0.00023269]
 [0.00191947 0.00184291 0.00311756 0.00033373]
 [0.00182549 0.00176139 0.00360656 0.00026072]

In [22]:
channels = ['F7', 'F5', 'F3', 'F1', 'F2', 'F4', 'F6', 'F8', 'P7', 'P5', 'P3', 'P1', 'P2', 'P4', 'P6', 'P8', 'O1', 'O2', 'C5', 'C3', 'C1', 'C2', 'C4', 'C6', 'T9', 'T7', 'T8', 'T10']
theta_columns = [f'{channel}_theta' for channel in channels]
alpha_columns = [f'{channel}_alpha' for channel in channels]
#delta_columns = [f'{channel}_delta' for channel in channels]
#beta_columns = [f'{channel}_beta' for channel in channels]

#columns = theta_columns + alpha_columns + delta_columns + beta_columns
columns = theta_columns + alpha_columns 

In [23]:
this_df_eeg = pd.DataFrame(columns=["participant_id"]+columns)
for sub_id in df['participant_id'].values:
    try:
        mat_epochs_readed = mne.read_epochs(f"{dir_with_data}{sub_id}-epo.fif")
        evoked = mat_epochs_readed.average()
        evoked_channels = evoked.pick_channels(channels)
        features = eeg_power_band(evoked_channels, FREQ_BANDS)
        features_flatten = features.flatten()
        feat_0 = {column: [feature[0]] for column, feature in zip(theta_columns, features)}
        feat_1 = {column: [feature[1]] for column, feature in zip(alpha_columns, features)}
        #feat_2 = {column: [feature[2]] for column, feature in zip(delta_columns, features)}
        #feat_3 = {column: [feature[3]] for column, feature in zip(beta_columns, features)}
        name = {"participant_id": [sub_id]}
        print('--------------------------------------------------------------------------------')
        print(feat_0)
        print(feat_1)
        #print(feat_2)
        #print(feat_3)
        print(name)
        #d = {**name, **feat_0, **feat_1, **feat_2, **feat_3}
        d = {**name, **feat_0, **feat_1}
        new_frame = pd.DataFrame(d)
        print(this_df_eeg)
        print(new_frame)
        this_df_eeg = pd.concat([this_df_eeg, new_frame])
    except:
        print("file for "+sub_id+" doesn't exist")

Reading c:\dev\python\eggML-project\..\ds003944\preprocessed-data-autoreject\sub-1448-epo.fif ...
    Found the data of interest:
        t =       0.00 ...   29999.00 ms
        0 CTF compensation matrices available
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
--------------------------------------------------------------------------------
{'F7_theta': [0.0014552992236053265], 'F5_theta': [0.0015444846994001444], 'F3_theta': [0.0017126678355486927], 'F1_theta': [0.001721945672737233], 'F2_theta': [0.0016457623356486385], 'F4_theta': [0.0016920656933027028], 'F6_theta': [0.0016353958914408179], 'F8_theta': [0.00149326204034543], 'P7_theta': [0.0009114542312519759], 'P5_theta': [0.001138272353390117], 'P3_theta': [0.001604115654034091], 'P1_theta': [0.0018446562854560168], 'P2_theta': [0.002061032113056077], 'P4_theta': [0.00216031123852008], 'P6_theta': [0.001842202

In [24]:
print(this_df_eeg)

   participant_id  F7_theta  F5_theta  F3_theta  F1_theta  F2_theta  F4_theta  \
0        sub-1448  0.001455  0.001544  0.001713  0.001722  0.001646  0.001692   
0        sub-1824  0.000900  0.000833  0.001009  0.001180  0.001170  0.001119   
0        sub-1971  0.000567  0.000837  0.000844  0.000878  0.000955  0.000853   
0        sub-1989  0.001565  0.001648  0.001705  0.001822  0.001846  0.001734   
0        sub-1990  0.001204  0.001247  0.001330  0.001356  0.001369  0.001449   
..            ...       ...       ...       ...       ...       ...       ...   
0       sub-2177A  0.000773  0.000813  0.000906  0.001081  0.001190  0.001027   
0       sub-2184A  0.001005  0.001262  0.001369  0.001387  0.001212  0.000990   
0       sub-2193A  0.000434  0.000692  0.000879  0.000948  0.000930  0.000718   
0       sub-2214A  0.001288  0.001287  0.001285  0.001278  0.001271  0.001274   
0       sub-2217A  0.000901  0.001065  0.001176  0.001227  0.001084  0.001024   

    F6_theta  F8_theta  P7_

In [25]:
this_df_eeg['type'] = this_df_eeg['participant_id'].map(df.set_index('participant_id')['type'])

In [26]:
this_df_eeg

,participant_id,F7_theta,F5_theta,F3_theta,F1_theta,F2_theta,F4_theta,F6_theta,F8_theta,P7_theta,...,C3_alpha,C1_alpha,C2_alpha,C4_alpha,C6_alpha,T9_alpha,T7_alpha,T8_alpha,T10_alpha,type
0,sub-1448,0.001455,0.001544,0.001713,0.001722,0.001646,0.001692,0.001635,0.001493,0.000911,...,0.001373,0.001155,0.000991,0.000860,0.001024,0.001099,0.001251,0.001670,0.001073,Control
0,sub-1824,0.000900,0.000833,0.001009,0.001180,0.001170,0.001119,0.001190,0.001433,0.001026,...,0.002281,0.002254,0.001991,0.001901,0.002081,0.002153,0.001989,0.001851,0.002050,Psychosis
0,sub-1971,0.000567,0.000837,0.000844,0.000878,0.000955,0.000853,0.000824,0.000744,0.000807,...,0.000516,0.000514,0.000542,0.000610,0.000613,0.000680,0.000689,0.000522,0.000811,Control
0,sub-1989,0.001565,0.001648,0.001705,0.001822,0.001846,0.001734,0.001724,0.001752,0.000973,...,0.003254,0.003348,0.003760,0.002973,0.003791,0.003697,0.003330,0.003232,0.003485,Control
0,sub-1990,0.001204,0.001247,0.001330,0.001356,0.001369,0.001449,0.001478,0.001425,0.001306,...,0.001035,0.000928,0.000950,0.000806,0.001184,0.001295,0.001620,0.001376,0.001333,Psychosis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,sub-2177A,0.000773,0.000813,0.000906,0.001081,0.001190,0.001027,0.000857,0.000836,0.000780,...,0.001835,0.001884,0.001959,0.001817,0.001656,0.001545,0.001542,0.002407,0.002265,Psychosis
0,sub-2184A,0.001005,0.001262,0.001369,0.001387,0.001212,0.000990,0.000826,0.000633,0.000983,...,0.003804,0.003483,0.002861,0.002473,0.002484,0.002438,0.002611,0.003990,0.003017,Psychosis
0,sub-2193A,0.000434,0.000692,0.000879,0.000948,0.000930,0.000718,0.000563,0.000457,0.000726,...,0.003144,0.003148,0.002839,0.003457,0.003595,0.003382,0.003344,0.004330,0.005179,Control
0,sub-2214A,0.001288,0.001287,0.001285,0.001278,0.001271,0.001274,0.001272,0.001272,0.001303,...,0.001289,0.001290,0.001292,0.001313,0.001323,0.001316,0.001311,0.001294,0.001289,Control


In [27]:
#create new data frame to work on 
new_df = this_df_eeg.copy()
channels = ['F', 'P','C']
frequencies = ['alpha', 'theta']
#frequencies = ['beta']
columns_to_keep = ['type', 'participant_id']



# select the theta and alpha columns for channels starting with "F"
for frequency in frequencies:
  new_df[f'F_average_{frequency}'] = new_df[[f'F{i}_{frequency}' for i in range (1,8)]].mean(axis=1)
  new_df[f'P_average_{frequency}'] = new_df[[f'P{i}_{frequency}' for i in range (1,8)]].mean(axis=1)
  new_df[f'O_average_{frequency}'] = new_df[[f'O{i}_{frequency}' for i in range (1,2)]].mean(axis=1)
  new_df[f'C_average_{frequency}'] = new_df[[f'C{i}_{frequency}' for i in range (1,6)]].mean(axis=1)
  new_df[f'T_average_{frequency}'] = new_df[[f'T{i}_{frequency}' for i in range (7,10)]].mean(axis=1)
  for channel in channels:
    columns_to_keep.append(f'{channel}_average_{frequency}')


'''f_theta_columns = ['F7_theta', 'F5_theta', 'F3_theta', 'F1_theta', 'F2_theta', 'F4_theta', 'F6_theta', 'F8_theta']
f_alpha_columns = ['F7_alpha', 'F5_alpha', 'F3_alpha', 'F1_alpha', 'F2_alpha', 'F4_alpha', 'F6_alpha', 'F8_alpha']
f_delta_columns = [f'F{i}_delta' for i <- 1...8]
p_theta_columns = ['P1_theta', 'P2_theta', 'P3_theta', 'P4_theta', 'P5_theta', 'P6_theta', 'P7_theta', 'P8_theta']
p_alpha_columns = ['P1_alpha', 'P2_alpha', 'P3_alpha', 'P4_alpha', 'P5_alpha', 'P6_alpha', 'P7_alpha', 'P8_alpha']
p_delta_columns = [f'P{i}_delta' for i <- 1...8]
o_theta_columns = ['O1_theta', 'O2_theta']
o_alpha_columns = ['O1_alpha', 'O2_alpha']
o_delta_columns = ['O1_delta', 'O2_delta']
c_theta_columns = ['C1_theta', 'C2_theta', 'C3_theta', 'C4_theta', 'C5_theta', 'C6_theta']
c_alpha_columns = ['C1_alpha', 'C2_alpha', 'C3_alpha', 'C4_alpha', 'C5_alpha', 'C6_alpha']
c_delta_columns = [f'C{i}_delta' for i <- 1...6]
t_theta_columns = ['T7_theta', 'T8_theta', 'T9_theta', 'T10_theta']
t_alpha_columns = ['T7_alpha', 'T8_alpha', 'T9_alpha', 'T10_alpha']
t_delta_columns = [f'T{i}_delta' for i <- 7...10]

# calculate the row-wise mean for the selected columns and rename the resulting columns
new_df['F_average_theta'] = new_df[f_theta_columns].mean(axis=1)
new_df['F_average_alpha'] = new_df[f_alpha_columns].mean(axis=1)
new_df['P_average_theta'] = new_df[p_theta_columns].mean(axis=1)
new_df['P_average_alpha'] = new_df[p_alpha_columns].mean(axis=1)
new_df['o_average_theta'] = new_df[o_theta_columns].mean(axis=1)
new_df['o_average_alpha'] = new_df[o_alpha_columns].mean(axis=1)
new_df['c_average_theta'] = new_df[c_theta_columns].mean(axis=1)
new_df['c_average_alpha'] = new_df[c_alpha_columns].mean(axis=1)
new_df['t_average_theta'] = new_df[t_theta_columns].mean(axis=1)
new_df['t_average_alpha'] = new_df[t_alpha_columns].mean(axis=1)
columns_to_keep = ['type', 'participant_id', 'P_average_theta', 'P_average_alpha', 'F_average_theta', 'F_average_alpha', 'o_average_theta', 'o_average_alpha', 'c_average_theta', 'c_average_alpha', 't_average_theta', 't_average_alpha']'''

print(columns_to_keep)
new_df = new_df.drop(columns=[col for col in new_df.columns if col not in columns_to_keep])

print(new_df)

['type', 'participant_id', 'F_average_alpha', 'P_average_alpha', 'C_average_alpha', 'F_average_theta', 'P_average_theta', 'C_average_theta']
   participant_id       type  F_average_alpha  P_average_alpha  \
0        sub-1448    Control         0.001007         0.001086   
0        sub-1824  Psychosis         0.001115         0.002028   
0        sub-1971    Control         0.000430         0.000493   
0        sub-1989    Control         0.001697         0.002705   
0        sub-1990  Psychosis         0.001016         0.000945   
..            ...        ...              ...              ...   
0       sub-2177A  Psychosis         0.000967         0.001029   
0       sub-2184A  Psychosis         0.001167         0.001613   
0       sub-2193A    Control         0.001083         0.002201   
0       sub-2214A    Control         0.001304         0.001310   
0       sub-2217A  Psychosis         0.000849         0.001148   

    C_average_alpha  F_average_theta  P_average_theta  C_average_t

In [28]:
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, balanced_accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

#użyty svr

# load data
data = new_df.copy()

# drop the participant_id column
data = data.drop(columns=['participant_id'])
# replace 'Psychosis' with 1 and 'Control' with 0 in the 'type' column
data['type'] = data['type'].replace({'Psychosis': 1, 'Control': 0})

# separate the target variable from the features
X = data.drop(columns=['type'])
y = data['type']
print(data)

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

print(f"Shape of the X train dataset: {X_train.shape}")
print(f"Shape of the X test dataset: {X_test.shape}")
print(f"Shape of the y train dataset: {y_train.shape}")
print(f"Shape of the y test dataset: {y_test.shape}")

# define the parameter grid to search over
param_grid = {'C': [0.1, 1, 10, 100], 'epsilon': [0.1, 0.2, 0.5, 1], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}
#k_range = list(range(1, 10))
#param_grid = dict(n_neighbors=k_range)

# create a Support Vector Regression model
model = SVR()
#model = KNeighborsClassifier()

# perform a grid search to find the best parameters
grid_search = GridSearchCV(model, param_grid, cv=10, scoring=['accuracy', 'roc_auc', 'precision', 'recall', 'balanced_accuracy'], refit='balanced_accuracy')
grid_search.fit(X_train, y_train)


    type  F_average_alpha  P_average_alpha  C_average_alpha  F_average_theta  \
0      0         0.001007         0.001086         0.001177         0.001630   
0      1         0.001115         0.002028         0.002143         0.001057   
0      0         0.000430         0.000493         0.000539         0.000823   
0      0         0.001697         0.002705         0.003290         0.001720   
0      1         0.001016         0.000945         0.000983         0.001347   
..   ...              ...              ...              ...              ...   
0      1         0.000967         0.001029         0.001832         0.000950   
0      1         0.001167         0.001613         0.003338         0.001150   
0      0         0.001083         0.002201         0.003112         0.000738   
0      0         0.001304         0.001310         0.001295         0.001279   
0      1         0.000849         0.001148         0.001725         0.001055   

    P_average_theta  C_average_theta  


GridSearchCV(cv=10, estimator=SVR(),
             param_grid={'C': [0.1, 1, 10, 100], 'epsilon': [0.1, 0.2, 0.5, 1],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             refit='balanced_accuracy',
             scoring=['accuracy', 'roc_auc', 'precision', 'recall',
                      'balanced_accuracy'])

In [13]:
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, balanced_accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

#użyty KNN

# load data
data = new_df.copy()

# drop the participant_id column
data = data.drop(columns=['participant_id'])
# replace 'Psychosis' with 1 and 'Control' with 0 in the 'type' column
data['type'] = data['type'].replace({'Psychosis': 1, 'Control': 0})

# separate the target variable from the features
X = data.drop(columns=['type'])
y = data['type']
print(data)

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

print(f"Shape of the X train dataset: {X_train.shape}")
print(f"Shape of the X test dataset: {X_test.shape}")
print(f"Shape of the y train dataset: {y_train.shape}")
print(f"Shape of the y test dataset: {y_test.shape}")

# define the parameter grid to search over
# param_grid = {'C': [0.1, 1, 10, 100], 'epsilon': [0.1, 0.2, 0.5, 1], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}
k_range = list(range(1, 10))
param_grid = dict(n_neighbors=k_range)

# create a Support Vector Regression model
#model = SVR()
model = KNeighborsClassifier()

# perform a grid search to find the best parameters
grid_search = GridSearchCV(model, param_grid, cv=10, scoring=['accuracy', 'roc_auc', 'precision', 'recall', 'balanced_accuracy'], refit='balanced_accuracy')
grid_search.fit(X_train, y_train)


    type  F_average_alpha  P_average_alpha  C_average_alpha  F_average_theta  \
0      0         0.001007         0.001086         0.001177         0.001630   
0      1         0.001115         0.002028         0.002143         0.001057   
0      0         0.000430         0.000493         0.000539         0.000823   
0      0         0.001697         0.002705         0.003290         0.001720   
0      1         0.001016         0.000945         0.000983         0.001347   
..   ...              ...              ...              ...              ...   
0      1         0.000967         0.001029         0.001832         0.000950   
0      1         0.001167         0.001613         0.003338         0.001150   
0      0         0.001083         0.002201         0.003112         0.000738   
0      0         0.001304         0.001310         0.001295         0.001279   
0      1         0.000849         0.001148         0.001725         0.001055   

    P_average_theta  C_average_theta  


GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9]},
             refit='balanced_accuracy',
             scoring=['accuracy', 'roc_auc', 'precision', 'recall',
                      'balanced_accuracy'])

In [29]:
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# make predictions on the testing data using the best model
y_pred = best_model.predict(X_test)

# convert the predicted values to binary format
y_pred_bin = [1 if x > 0.5 else 0 for x in y_pred]

# convert the actual values to binary format
y_test_bin = [1 if x > 0.5 else 0 for x in y_test]

# evaluate the performance of the best model using different metrics
accuracy = accuracy_score(y_test_bin, y_pred_bin)
roc_auc = roc_auc_score(y_test_bin, y_pred_bin)
precision = precision_score(y_test_bin, y_pred_bin)
recall = recall_score(y_test_bin, y_pred_bin)
balanced_accuracy = balanced_accuracy_score(y_test_bin, y_pred_bin)

# print out the best parameters and the evaluation metrics for the best model
print("Best parameters:", best_params)
print("Accuracy:", accuracy)
print("ROC AUC:", roc_auc)
print("Precision:", precision)
print("Recall:", recall)
print("Balanced accuracy:", balanced_accuracy)

Best parameters: {'C': 0.1, 'epsilon': 0.1, 'kernel': 'linear'}
Accuracy: 0.75
ROC AUC: 0.5
Precision: 0.75
Recall: 1.0
Balanced accuracy: 0.5


In [30]:
print(y_pred)
print(y_pred_bin)

[0.89999897 0.89999965 0.90000016 0.89999993 0.89999879 0.89999971
 0.89999992 0.90000009 0.90000032 0.89999948 0.8999989  0.90000018
 0.8999994  0.90000003 0.89999905 0.89999971 0.90000007 0.89999867
 0.90000004 0.89999919 0.9000001  0.89999777 0.89999963 0.89999892
 0.89999953 0.89999959 0.89999999 0.89999851 0.89999855 0.89999886
 0.89999975 0.89999922]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
